In [104]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [117]:
cd '/content/drive/MyDrive/Eam/Analitica_predictiva/DespliegueEAM'

/content/drive/MyDrive/Eam/Analitica_predictiva/DespliegueEAM


In [118]:
import pandas as pd
# Load the second sheet of the Excel file
df_sheet2 = pd.read_excel('/content/drive/MyDrive/Eam/Analitica_predictiva/Aprobacion curso 2019 fut.xlsx', sheet_name=1)
display(df_sheet2.head())

,ID,Año - Semestre,Felder,Examen_admisión_Universidad
0,161,2019-01,activo,4.00
1,162,2019-01,visual,3.67
2,163,2019-01,equilibrio,4.50
3,164,2019-01,visual,3.00
4,165,2019-01,equilibrio,4.67


In [119]:
df_sheet2 = df_sheet2.drop(['ID', 'Año - Semestre'], axis=1)
display(df_sheet2.head())

,Felder,Examen_admisión_Universidad
0,activo,4.00
1,visual,3.67
2,equilibrio,4.50
3,visual,3.00
4,equilibrio,4.67


In [120]:
import joblib

onehotencoder = joblib.load('onehot_encoder.pkl')
standard_scaler = joblib.load('minmax_scaler.pkl')
best_bagging_linear_regressor_model = joblib.load('best_model.pkl')

print("Loaded objects as separate variables:")
print(f"- onehotencoder: {type(onehotencoder)}")
print(f"- standard_scaler: {type(standard_scaler)}")
print(f"- best_bagging_linear_regressor_model: {type(best_bagging_linear_regressor_model)}")

Loaded objects as separate variables:
- onehotencoder: <class 'sklearn.preprocessing._encoders.OneHotEncoder'>
- standard_scaler: <class 'sklearn.preprocessing._data.MinMaxScaler'>
- best_bagging_linear_regressor_model: <class 'sklearn.ensemble._bagging.BaggingClassifier'>


In [121]:
df_sheet2.columns

Index(['Felder', 'Examen_admisión_Universidad'], dtype='object')

In [122]:
import numpy as np

# Select the 'Examen_admisión' column and reshape it for the scaler
examen_admision = df_sheet2['Examen_admisión_Universidad'].values.reshape(-1, 1)

# Apply the standard_scaler
df_sheet2['Examen_admisión_Universidad'] = standard_scaler.transform(examen_admision)

display(df_sheet2.head())

,Felder,Examen_admisión_Universidad
0,activo,0.500
1,visual,0.335
2,equilibrio,0.750
3,visual,0.000
4,equilibrio,0.835


In [123]:
# Apply one-hot encoding to the 'Felder' column
felder_encoded = onehotencoder.transform(df_sheet2[['Felder']])

# Convert the encoded array to a DataFrame with appropriate column names
# Assuming the onehotencoder has get_feature_names_out method
try:
    feature_names = onehotencoder.get_feature_names_out(['Felder'])
except AttributeError:
    # Fallback if get_feature_names_out is not available (older versions)
    # This part might need adjustment based on the actual encoder fitted
    feature_names = [f'Felder_{cat}' for cat in onehotencoder.categories_[0]]

felder_encoded_df = pd.DataFrame(felder_encoded, columns=feature_names, index=df_sheet2.index)

# Concatenate the encoded features with the original DataFrame (excluding the original 'Felder' column)
df_sheet2 = pd.concat([df_sheet2.drop('Felder', axis=1), felder_encoded_df], axis=1)

display(df_sheet2.head())

,Examen_admisión_Universidad,Felder_activo,Felder_equilibrio,Felder_intuitivo,Felder_reflexivo,Felder_secuencial,Felder_sensorial,Felder_verbal,Felder_visual
0,0.500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.335,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.750,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.835,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
# Rename the column 'Examen_admisión_Universidad' to 'Examen_admisión_Universidad_scaled'
df_sheet2 = df_sheet2.rename(columns={'Examen_admisión_Universidad': 'Examen_admisión_Universidad_scaled'})

# Move the renamed column to the end
scaled_column = df_sheet2.pop('Examen_admisión_Universidad_scaled')
df_sheet2['Examen_admisión_Universidad_scaled'] = scaled_column

display(df_sheet2.head())

,Felder_activo,Felder_equilibrio,Felder_intuitivo,Felder_reflexivo,Felder_secuencial,Felder_sensorial,Felder_verbal,Felder_visual,Examen_admisión_Universidad_scaled
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.335
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.835


In [127]:
# Make predictions using the loaded model
predictions = best_bagging_linear_regressor_model.predict(df_sheet2)

# Add the predictions to the DataFrame
df_sheet2['Predictions'] = predictions

display(df_sheet2.head())

,Felder_activo,Felder_equilibrio,Felder_intuitivo,Felder_reflexivo,Felder_secuencial,Felder_sensorial,Felder_verbal,Felder_visual,Examen_admisión_Universidad_scaled,Predictions
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500,si
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.335,no
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.750,si
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000,no
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.835,si


In [128]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.3 MB/s eta 0:00:00


# Task
Create a streamlit application that includes all the previous steps.

## Create a streamlit script

### Subtask:
Generate a Python file that will contain the code for the Streamlit application.


**Reasoning**:
Create a new Python file named `app.py` and write the initial Streamlit application code into it, including necessary imports and loading the saved models.



In [129]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib
import numpy as np

st.title('Course Approval Prediction')

# Load the trained models and encoders
try:
    onehotencoder = joblib.load('onehot_encoder.pkl')
    standard_scaler = joblib.load('minmax_scaler.pkl')
    best_bagging_linear_regressor_model = joblib.load('best_model.pkl')
except FileNotFoundError:
    st.error("Error: Model files not found. Please make sure 'onehot_encoder.pkl', 'minmax_scaler.pkl', and 'best_model.pkl' are in the same directory.")
    st.stop()

st.write("Please provide the following information to predict course approval.")

# Get user inputs
felder_options = ['activo', 'visual', 'equilibrio', 'intuitivo', 'reflexivo', 'secuencial', 'sensorial', 'verbal']
selected_felder = st.selectbox('Felder Learning Style', felder_options)
examen_admision = st.number_input('University Admission Exam Score', min_value=0.0, max_value=5.0, step=0.01)

# Create a DataFrame from user inputs
input_data = pd.DataFrame({
    'Felder': [selected_felder],
    'Examen_admisión_Universidad': [examen_admision]
})

# Preprocess the input data
# Scale the numerical feature
input_data['Examen_admisión_Universidad'] = standard_scaler.transform(input_data[['Examen_admisión_Universidad']])

# Apply one-hot encoding to the 'Felder' column
try:
    felder_encoded = onehotencoder.transform(input_data[['Felder']])
    feature_names = onehotencoder.get_feature_names_out(['Felder'])
except AttributeError:
    feature_names = [f'Felder_{cat}' for cat in onehotencoder.categories_[0]]

felder_encoded_df = pd.DataFrame(felder_encoded, columns=feature_names, index=input_data.index)

# Concatenate the encoded features with the original DataFrame (excluding the original 'Felder' column)
processed_input_data = pd.concat([input_data.drop('Felder', axis=1), felder_encoded_df], axis=1)

# Rename the scaled column to match the training data column name
processed_input_data = processed_input_data.rename(columns={'Examen_admisión_Universidad': 'Examen_admisión_Universidad_scaled'})

# Ensure the columns are in the same order as the training data
# Assuming the training data columns order can be retrieved from the loaded model or a saved list
# For this example, we'll manually list the expected order based on previous steps
expected_columns = ['Felder_activo', 'Felder_equilibrio', 'Felder_intuitivo', 'Felder_reflexivo',
                    'Felder_secuencial', 'Felder_sensorial', 'Felder_verbal', 'Felder_visual',
                    'Examen_admisión_Universidad_scaled']

processed_input_data = processed_input_data.reindex(columns=expected_columns, fill_value=0)


# Make prediction
if st.button('Predict'):
    prediction = best_bagging_linear_regressor_model.predict(processed_input_data)

    st.subheader('Prediction Result')
    if prediction[0] == 'si':
        st.success('The student is predicted to approve the course.')
    else:
        st.error('The student is predicted to not approve the course.')

Writing app.py


## Provide instructions to run the app

### Subtask:
Provide instructions to the user on how to save and run the created Streamlit application script.


## Summary:

### Data Analysis Key Findings

*   A Streamlit application script named `app.py` was successfully created to predict course approval based on user input.
*   The application loads pre-trained machine learning models and encoders (`onehot_encoder.pkl`, `minmax_scaler.pkl`, `best_model.pkl`) to perform predictions.
*   User inputs for 'Felder Learning Style' and 'University Admission Exam Score' are collected via Streamlit widgets.
*   The input data is preprocessed by scaling the exam score and one-hot encoding the Felder style before being fed into the model.
*   The application provides a clear prediction of whether a student is likely to approve the course based on the model's output.
*   Instructions were provided on how to save the script as `app.py` and run the Streamlit application using the command `streamlit run app.py`.

### Insights or Next Steps

*   Ensure the required model files (`onehot_encoder.pkl`, `minmax_scaler.pkl`, and `best_model.pkl`) are accessible in the same directory as `app.py` for the application to function correctly.
*   Consider deploying the Streamlit application to a hosting platform like Streamlit Cloud or Heroku to make it accessible to a wider audience.
